# Crowd Detection Project - Image Exporter

This process generates training and testing data for image classification from video feeds given ground-truth data found in a feature class.

## Pseudocode

1. Import needed modules

2. Establish runtime variables
    - GIS
    - video root directory
    - labels feature class
    - labels FC camera field
    - labels FC label field
    - labels FC datetime field
    
3. Create Helper Functions:
    - Input: label FC record (datetime, label, camera) | Output: video url and timestamp

4. Iteration: For each record of the labeled feature class...
    - Run helper function to determine video url and timestamp
    - Determine output path
    - Run export using moviepy

### 1. Import needed modules

In [21]:
import arcgis
import moviepy.editor as mpy
import datetime
import os

### 2. Establish runtime variables

- GIS
- video root directory
- labels feature class
- labels FC camera field
- labels FC label field
- labels FC datetime field

In [172]:
gis = arcgis.gis.GIS("https://esrifederal.maps.arcgis.com", username="Anieto_esrifederal")

Enter password: ········


In [173]:
# Make reference to directories and other needed paths
video_root_dir = r"D:\5_Data\Transportation\Cameras\SunTrustPark"
labels_fc = r"D:\3_Sandbox_Projects\1806_CobbCounty_PedestrianDetection\Inputs\S123_Labels.gdb\PedestrianCount"
labels_fc_camera_field = "CameraID"
labels_fc_label_field = "degreesaturation"
labels_fc_datetime_field = "PhotoDateTime"

### UnitTests

In [175]:
ix = 0

In [176]:
ts = labels_sdf.iloc[ix]['PhotoDateTime']
ts

Timestamp('2018-06-13 01:58:00')

In [177]:
camera = labels_sdf.iloc[ix]['CameraID']
camera

'129'

In [178]:
pdt = ts.to_pydatetime()
pdt

datetime.datetime(2018, 6, 13, 1, 58)

In [179]:
pdt = pdt - datetime.timedelta(hours=4)
pdt

datetime.datetime(2018, 6, 12, 21, 58)

In [180]:
yymmdd = pdt.strftime("%y%m%d")
yymmdd

'180612'

In [181]:
hh = pdt.strftime("%H")
hh

'21'

In [182]:
mm = pdt.strftime("%M")
mm

'58'

In [183]:
ss = pdt.strftime("%S")
ss

'00'

In [184]:
video_files = os.listdir("{0}\\{1}\\{2}".format(video_root_dir, yymmdd, camera))
video_files

['21h39m58s.avi', '22h29m30s.avi', '23h21m50s.avi']

### 3. Create Helper Functions

In [185]:
# Input: List of video files with names HHhMMmSSs.avi, datetime object | Output: Video containing the datetime
def get_videopath_and_video_timestamp_from_label_timestamp(video_files, datetime_object):
    
    # For each file in video files...
    for ix, video_file in enumerate(video_files):
    
        # we need a datetime range...
        start = datetime.datetime.strptime(video_files[ix]+" "+yymmdd, "%Hh%Mm%Ss.avi %y%m%d")
        end = datetime.datetime.strptime(video_files[ix+1]+" "+yymmdd, "%Hh%Mm%Ss.avi %y%m%d")
        in_time_range = start <= datetime_object <= end
        
        # so we can determine if the current timeslot exists there
        if in_time_range:
            
            # And get the video timestamp
            delta_video_timestamp = datetime_object - start
           
            return video_file, str(delta_video_timestamp)

In [186]:
# Unittest for get_videopath_and_video_timestamp_from_label_timestamp
get_videopath_and_video_timestamp_from_label_timestamp(video_files, pdt)

('21h39m58s.avi', '0:18:02')

In [187]:
# Input: label FC record (datetime, label, camera) | Output: video url and timestamp
def retrieve_video_and_time(camera, 
                            label, 
                            pandas_timestamp,
                            video_root_dir):
    
    # This function relies on a video root directory structure as follows:
    
    # Root_Path
        # yymmdd
            # Camera ID
                # HHhMMmSSs.avi
                
    # Convert pandas timestamp to python datetime object
    pdt = pandas_timestamp.to_pydatetime()
    pdt = pdt - datetime.timedelta(hours=4)
    print(pdt)
    
    # Parse out datetime object into needed formats
    yymmdd = pdt.strftime("%y%m%d")
    hh = pdt.strftime("%H")
    mm = pdt.strftime("%M")
    ss = pdt.strftime("%S")
    
    # Determine which video contains the needed time and at which time the label was recorded in the video
    camera_dir = "{0}\\{1}\\{2}".format(video_root_dir, yymmdd, camera)
    video_files = os.listdir(camera_dir)
    video_name, video_time = get_videopath_and_video_timestamp_from_label_timestamp(video_files, pdt)
    
    return "{0}\\{1}".format(camera_dir, video_name), video_time

In [188]:
ts

Timestamp('2018-06-13 01:58:00')

In [189]:
retrieve_video_and_time(camera, "High", ts, video_root_dir)

2018-06-12 21:58:00


('D:\\5_Data\\Transportation\\Cameras\\SunTrustPark\\180612\\129\\21h39m58s.avi',
 '0:18:02')

### 4.Iteration: For each record of the labeled feature class...
    - Run helper function to determine video url and timestamp
    - Determine output path
    - Run export using moviepy

In [195]:
def main(label_fc, labels_fc_camera_field, labels_fc_label_field, labels_fc_datetime_field, video_root_dir):
    # Convert the labels fc into a spatial dataframe in python memory
    labels_sdf = arcgis.features.SpatialDataFrame.from_featureclass(labels_fc)
    # Iterate on each record to start the image export
    for index, row in labels_sdf.iterrows():
        
        camera = row[labels_fc_camera_field] 
        label = row[labels_fc_label_field]
        pandas_timestamp = row[labels_fc_datetime_field]
        
        print("\n<<<< >>>>")
        print(camera)
        print(label)
        print(pandas_timestamp)
        print("\n")
                
        # Get the path to the right video and the timestamp in the video for the recorded label
        video_path, video_time =  retrieve_video_and_time(camera, label, pandas_timestamp, video_root_dir)
        print(video_path, video_time)
        
#         video_path = r"D:\5_Data\Transportation\Cameras\SunTrustPark\180612\128\18h57m39s.avi"
#         clip = mpy.VideoFileClip(video_path)
#         clip.save_frame("testframe.png")
#         clip.save_frame("testframe02.png", t='00:28:55')

In [196]:
# Make reference to directories and other needed paths
video_root_dir = r"D:\5_Data\Transportation\Cameras\SunTrustPark"
labels_fc = r"D:\3_Sandbox_Projects\1806_CobbCounty_PedestrianDetection\Inputs\S123_Labels.gdb\PedestrianCount"
labels_fc_camera_field = "CameraID"
labels_fc_label_field = "degreesaturation"
labels_fc_datetime_field = "PhotoDateTime"

# Run main
main(labels_fc, labels_fc_camera_field, labels_fc_label_field, labels_fc_datetime_field, video_root_dir)


<<<< >>>>
129
Medium
2018-06-13 01:58:00


2018-06-12 21:58:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:18:02

<<<< >>>>
130
Low
2018-06-13 01:59:00


2018-06-12 21:59:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\21h15m32s.avi 0:43:28

<<<< >>>>
129
Medium
2018-06-13 02:00:00


2018-06-12 22:00:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:20:02

<<<< >>>>
129
Medium
2018-06-13 02:01:00


2018-06-12 22:01:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:21:02

<<<< >>>>
130
Low
2018-06-13 02:01:00


2018-06-12 22:01:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\21h15m32s.avi 0:45:28

<<<< >>>>
129
Medium
2018-06-13 02:02:00


2018-06-12 22:02:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:22:02

<<<< >>>>
129
Medium
2018-06-13 02:04:00


2018-06-12 22:04:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:24:02

<<<<

2018-06-12 22:28:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\21h39m58s.avi 0:48:02

<<<< >>>>
129
High
2018-06-13 02:30:00


2018-06-12 22:30:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\22h29m30s.avi 0:00:30

<<<< >>>>
129
High
2018-06-13 02:30:00


2018-06-12 22:30:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\22h29m30s.avi 0:00:30

<<<< >>>>
130
High
2018-06-13 02:31:00


2018-06-12 22:31:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\22h28m30s.avi 0:02:30

<<<< >>>>
130
High
2018-06-13 02:31:00


2018-06-12 22:31:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\22h28m30s.avi 0:02:30

<<<< >>>>
130
High
2018-06-13 02:32:00


2018-06-12 22:32:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\22h28m30s.avi 0:03:30

<<<< >>>>
129
High
2018-06-13 02:32:00


2018-06-12 22:32:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\129\22h29m30s.avi 0:02:30

<<<< >>>>
129
High
2018-06-13 02:34:00


2018-06-12 22

D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\21h15m32s.avi 0:38:28.000001

<<<< >>>>
130
Low
2018-06-13 01:53:00


2018-06-12 21:53:00
D:\5_Data\Transportation\Cameras\SunTrustPark\180612\130\21h15m32s.avi 0:37:28

<<<< >>>>
130
Low
2018-06-12 22:50:00


2018-06-12 18:50:00


IndexError: list index out of range